OpenClassrooms
Project 4, Data Scientist
Author : Oumeima EL GHARBI
Date : August 2022

Un notebook pour chaque prédiction (émissions de CO2 et consommation totale d’énergie) des différents tests de modèles mis au propre, dans lequel vous identifierez clairement le modèle final choisi.


L’objectif est de te passer des relevés de consommation annuels futurs (attention à la fuite de données). Nous ferons de toute façon pour tout nouveau bâtiment un premier relevé de référence la première année, donc rien ne t'interdit d’en déduire des variables structurelles aux bâtiments, par exemple la nature et proportions des sources d’énergie utilisées..

Fais bien attention au traitement des différentes variables, à la fois pour trouver de nouvelles informations (peut-on déduire des choses intéressantes d’une simple adresse ?) et optimiser les performances en appliquant des transformations simples aux variables (normalisation, passage au log, etc.).

Mets en place une évaluation rigoureuse des performances de la régression, et optimise les hyperparamètres et le choix d’algorithmes de ML à l’aide d’une validation croisée.



Tester les modèles suivants : **regression linéaire (avec différentes régularisation : Ridge, Lasso, Elastic), Random Forest, XGboost**
Penser à comparer les performances des différents modèles : utiliser la **MAE** ( Mean Absolute Error)
Penser également à optimiser les hyper paramètres de chaque modèle via **GridSearch**


Evaluate :

https://cloud.google.com/automl-tables/docs/evaluate?hl=fr


KFOLD

Entrée : données X (dimension nxp), étiquettes y (dimension n), nombre de folds k

Couper [0, 1, ..., n-1] en k parties de taille (n/k). (La dernière partie sera un peu plus petite si n n'est pas un multiple de k)

for i=0 to (k-1):
    Former le jeu de test (X_test, y_test) en restreignant X et y aux indices contenus dans la i-ième partie.
    Former le jeu d'entraînement (X_train, y_train) en restreignant X et y aux autres indices.
    Entraîner l'algorithme sur le jeu d'entraînement
    Utiliser le modèle ainsi obtenu pour prédire sur le jeu de test
        Calculer l'erreur du modèle en comparant les étiquettes prédites aux vraies étiquettes contenues dans y_test

Sortie : la valeur moyenne des erreurs calculées sur les k folds.

In [305]:
# 1 modele de Regression (RL classsque / Elastic / ridig / laso
# Random Forest
# XGBOOST

# var à rpedire tottal GHE Emssion last one to predict
# cette var deped de la consommation des bateimenst (1) prediction sur elec, steam, naturalgas et un autre energie (2) et reutiliser pour predire

### Introduction

#### Importing libraries

In [306]:
%reset -f

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%autosave 300

Autosaving every 300 seconds


In [307]:
from functions import *

In [308]:
import os, warnings
# warnings.filterwarnings('ignore')
warnings.filterwarnings(action="ignore")

from exploration import *

In [309]:
# Set random integer (seed) - 891
from random import randrange
#seed = randrange(1000)
seed = 1
print("Seed value: ", seed)

Seed value:  1


#### Loading dataset

In [310]:
global columns_to_categorize
columns_to_categorize = ["BuildingType", "PrimaryPropertyType", "Neighborhood"]
category_types = {column: 'object' for column in columns_to_categorize}


dataset = pd.read_csv(r"C:\Users\oumei\Documents\OC_projets\P4\P4_Data_Science_OpenClassrooms\dataset\cleaned\data_2016_tryout_2.csv", encoding='utf-8', dtype=category_types)

In [311]:
dataset.shape

(3147, 16)

In [312]:
dataset.dtypes

Neighborhood                            object
BuildingType                            object
PrimaryPropertyType                     object
ENERGYSTARScore                        float64
YearBuilt                                int64
Log-NumberofBuildings                  float64
Log-NumberofFloors                     float64
Log-PropertyGFATotal                   float64
Log-PropertyGFAParking                 float64
Log-SecondLargestPropertyUseTypeGFA    float64
Log-ThirdLargestPropertyUseTypeGFA     float64
Log-TotalEnergy(kBtu)                  float64
Log-SteamUse(kBtu)                     float64
Log-Electricity(kBtu)                  float64
Log-NaturalGas(kBtu)                   float64
Log-TotalGHGEmissions                  float64
dtype: object

In [313]:
# predict : Electricity
# train / test
# standardisation = retirer la moyen et div par ecart type (scaling : les var qn sur emem echelle
# var categ : encoding (one hot encoder)

# la fin Feature engineriing

# 2) entrainer le smodels
# perf
# temps de calcul
# graph pour montrer la perf de chaque modele(barplot)
# obj : finir exploration / finir feature engineering
# obj un premier noteboook propre (try max)

In [314]:
dataset.columns

Index(['Neighborhood', 'BuildingType', 'PrimaryPropertyType',
       'ENERGYSTARScore', 'YearBuilt', 'Log-NumberofBuildings',
       'Log-NumberofFloors', 'Log-PropertyGFATotal', 'Log-PropertyGFAParking',
       'Log-SecondLargestPropertyUseTypeGFA',
       'Log-ThirdLargestPropertyUseTypeGFA', 'Log-TotalEnergy(kBtu)',
       'Log-SteamUse(kBtu)', 'Log-Electricity(kBtu)', 'Log-NaturalGas(kBtu)',
       'Log-TotalGHGEmissions'],
      dtype='object')

### TRYOUT

In [315]:
#features_for_prediction = ["YearBuilt", "BuildingType", "PrimaryPropertyType", "Neighborhood", "Log-NumberofBuildings", "Log-NumberofFloors", "Log-PropertyGFATotal", "Log-PropertyGFAParking", "Log-SecondLargestPropertyUseTypeGFA", "Log-ThirdLargestPropertyUseTypeGFA", "Electricity_ratio"]

#features_for_prediction = ["YearBuilt", "NumberofBuildings", "NumberofFloors", "LargestPropertyUseTypeGFA", "SecondLargestPropertyUseTypeGFA", "ThirdLargestPropertyUseTypeGFA",
   #                        "BuildingType","PrimaryPropertyType", "Neighborhood", "LargestPropertyUseType", "SecondLargestPropertyUseType", "ThirdLargestPropertyUseType"]

#variable_to_predict = "Log-Electricity(kBtu)"

#features_for_prediction.append(variable_to_predict)
#print(features_for_prediction)


In [316]:
data = dataset.drop(['ENERGYSTARScore'], axis=1).copy()

In [317]:
# liste des variables et étiquettes
model_CO2_target = ['Log-TotalGHGEmissions']
model_CO2_features = ['Log-SteamUse(kBtu)', 'Log-Electricity(kBtu)', 'Log-NaturalGas(kBtu)', 'Log-TotalEnergy(kBtu)']
model_CO2 = model_CO2_features + model_CO2_target

In [318]:
# Liste des variables et étiquettes pour la modélisation
model_EnerStar_target = ['Log-TotalGHGEmissions']
model_EnerStar_features = model_CO2_features + ['ENERGYSTARScore']

model_EnerStar = model_EnerStar_features + model_EnerStar_target

In [319]:
# Liste des variables et étiquettes
models_energy_target = model_CO2_features
models_energy_features = [var for var in dataset.columns if var not in ['ENERGYSTARScore', *model_CO2_target, *model_CO2_features]]
models_energy = models_energy_features + models_energy_target

## I) Feature Engineering : preparing the vectors and matrices


#### 1) Separating training data and target vector

#### 2) Separation train and test dataset


In [320]:
print("We have to separate the train / test sets before normalising the dataset.")

We have to separate the train / test sets before normalising the dataset.


In [321]:
# We create a training set and a test set (the test set contains 30% of the dataset)
from sklearn. model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,  random_state=seed)

train, test = train_test_split(data, test_size=0.3,  random_state=seed)

In [322]:
train.shape

(2202, 15)

In [323]:
test.shape

(945, 15)

#### 3) Normalization

In [324]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [325]:
print("We separate categorical variables from numerical variables.")

We separate categorical variables from numerical variables.


In [326]:
data.select_dtypes(['category','object']) # we don't have 'object' here but it is just in case.

categorical_columns = data.select_dtypes(['category','object']).columns
numerical_columns = data[models_energy_features].select_dtypes(include='number').columns.drop("YearBuilt") ###
#numerical_columns = X.select_dtypes(include='number').columns
print("We won't normalise the year so we drop it from numerical_columns.")

print("Shape of categorical variables : ", categorical_columns.shape)
print("Shape of numerical variables :", numerical_columns.shape)

We won't normalise the year so we drop it from numerical_columns.
Shape of categorical variables :  (3,)
Shape of numerical variables : (6,)


##### 1) Data Standardisation

We have to standardize the variables before learning a **Ridge Regression**.
Standardizing means that each variable will have a **standard deviation** equal to 1.

In [327]:
print("Numerical variables standardization")
print("We have :", numerical_columns.shape[0], "numerical features to standardize.",end="\n\n")

print(numerical_columns)

Numerical variables standardization
We have : 6 numerical features to standardize.

Index(['Log-NumberofBuildings', 'Log-NumberofFloors', 'Log-PropertyGFATotal',
       'Log-PropertyGFAParking', 'Log-SecondLargestPropertyUseTypeGFA',
       'Log-ThirdLargestPropertyUseTypeGFA'],
      dtype='object')


In [328]:
# We train / fit the scaler on the training set / Computes the mean and std to be used for later scaling.
std_scale = StandardScaler().fit(train[numerical_columns])
# We transform the training set and the testing set / Performs standardization by centering and scaling.
train_std = train.copy()
test_std = test.copy()

train_std[numerical_columns] = std_scale.transform(train[numerical_columns])
test_std[numerical_columns] = std_scale.transform(test[numerical_columns])

print("Before")
display(train)
print("After")
display(train_std)

Before


,Neighborhood,BuildingType,PrimaryPropertyType,YearBuilt,Log-NumberofBuildings,Log-NumberofFloors,Log-PropertyGFATotal,Log-PropertyGFAParking,Log-SecondLargestPropertyUseTypeGFA,Log-ThirdLargestPropertyUseTypeGFA,Log-TotalEnergy(kBtu),Log-SteamUse(kBtu),Log-Electricity(kBtu),Log-NaturalGas(kBtu),Log-TotalGHGEmissions
1008,Northeast,Multifamily lr (1-4),Low-rise multifamily,1997,0.693147,1.386294,10.727246,0.0,9.792444,7.848153,14.067857,0.0,14.067857,0.000000,2.299581
1784,North,Multifamily lr (1-4),Low-rise multifamily,1979,0.693147,1.098612,10.429783,0.0,6.240276,0.000000,13.734070,0.0,13.734069,0.000000,2.005526
1563,Lake union,Multifamily lr (1-4),Low-rise multifamily,1988,0.693147,1.609438,10.120533,0.0,0.000000,0.000000,13.380600,0.0,13.380599,0.000000,1.706565
452,Magnolia / queen anne,Nonresidential,Small- and mid-sized office,1982,0.693147,1.791759,11.499526,0.0,0.000000,0.000000,15.295147,0.0,15.295147,0.000000,3.453474
1561,Downtown,Multifamily hr (10+),High-rise multifamily,1980,0.693147,2.944439,11.738139,0.0,9.063811,0.000000,15.389618,0.0,14.915060,14.416334,4.777357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2763,East,Multifamily mr (5-9),Mid-rise multifamily,1985,0.693147,1.791759,10.556620,0.0,0.000000,0.000000,14.117894,0.0,14.117894,0.000000,2.344686
905,Southwest,Nonresidential,Small- and mid-sized office,1981,0.693147,1.386294,10.677938,0.0,0.000000,0.000000,14.803849,0.0,14.671030,12.719411,3.559625
1096,Northwest,Multifamily lr (1-4),Low-rise multifamily,1978,2.833213,1.386294,11.397167,0.0,8.752423,0.000000,14.907515,0.0,14.907515,0.000000,3.080992
235,Downtown,Nonresidential,Small- and mid-sized office,1906,0.693147,1.609438,11.059441,0.0,0.000000,0.000000,15.073534,0.0,14.702697,13.901848,4.328494


After


,Neighborhood,BuildingType,PrimaryPropertyType,YearBuilt,Log-NumberofBuildings,Log-NumberofFloors,Log-PropertyGFATotal,Log-PropertyGFAParking,Log-SecondLargestPropertyUseTypeGFA,Log-ThirdLargestPropertyUseTypeGFA,Log-TotalEnergy(kBtu),Log-SteamUse(kBtu),Log-Electricity(kBtu),Log-NaturalGas(kBtu),Log-TotalGHGEmissions
1008,Northeast,Multifamily lr (1-4),Low-rise multifamily,1997,-0.035067,-0.287309,-0.238103,-0.405348,1.108652,1.972318,14.067857,0.0,14.067857,0.000000,2.299581
1784,North,Multifamily lr (1-4),Low-rise multifamily,1979,-0.035067,-0.782088,-0.582698,-0.405348,0.371487,-0.439239,13.734070,0.0,13.734069,0.000000,2.005526
1563,Lake union,Multifamily lr (1-4),Low-rise multifamily,1988,-0.035067,0.096471,-0.940947,-0.405348,-0.923528,-0.439239,13.380600,0.0,13.380599,0.000000,1.706565
452,Magnolia / queen anne,Nonresidential,Small- and mid-sized office,1982,-0.035067,0.410042,0.656541,-0.405348,-0.923528,-0.439239,15.295147,0.0,15.295147,0.000000,3.453474
1561,Downtown,Multifamily hr (10+),High-rise multifamily,1980,-0.035067,2.392512,0.932961,-0.405348,0.957442,-0.439239,15.389618,0.0,14.915060,14.416334,4.777357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2763,East,Multifamily mr (5-9),Mid-rise multifamily,1985,-0.035067,0.410042,-0.435764,-0.405348,-0.923528,-0.439239,14.117894,0.0,14.117894,0.000000,2.344686
905,Southwest,Nonresidential,Small- and mid-sized office,1981,-0.035067,-0.287309,-0.295223,-0.405348,-0.923528,-0.439239,14.803849,0.0,14.671030,12.719411,3.559625
1096,Northwest,Multifamily lr (1-4),Low-rise multifamily,1978,10.208154,-0.287309,0.537964,-0.405348,0.892821,-0.439239,14.907515,0.0,14.907515,0.000000,3.080992
235,Downtown,Nonresidential,Small- and mid-sized office,1906,-0.035067,0.096471,0.146726,-0.405348,-0.923528,-0.439239,15.073534,0.0,14.702697,13.901848,4.328494


In [329]:
print("We can check that the numerical variables have a Standard Normal distribution.")
#densite(train[numerical_columns])

We can check that the numerical variables have a Standard Normal distribution.


##### 2) Feature Encoding : One Hot Encoder

In [330]:
print("Categorical variables featuring")

print("We have :", categorical_columns.shape[0], "categorical features to encode.", end="\n\n")
print(categorical_columns)

Categorical variables featuring
We have : 3 categorical features to encode.

Index(['Neighborhood', 'BuildingType', 'PrimaryPropertyType'], dtype='object')


In [331]:
train.dtypes # we check that we have categories

Neighborhood                            object
BuildingType                            object
PrimaryPropertyType                     object
YearBuilt                                int64
Log-NumberofBuildings                  float64
Log-NumberofFloors                     float64
Log-PropertyGFATotal                   float64
Log-PropertyGFAParking                 float64
Log-SecondLargestPropertyUseTypeGFA    float64
Log-ThirdLargestPropertyUseTypeGFA     float64
Log-TotalEnergy(kBtu)                  float64
Log-SteamUse(kBtu)                     float64
Log-Electricity(kBtu)                  float64
Log-NaturalGas(kBtu)                   float64
Log-TotalGHGEmissions                  float64
dtype: object

In [332]:
train[categorical_columns].nunique()

Neighborhood           13
BuildingType            7
PrimaryPropertyType    23
dtype: int64

In [333]:
train_std[categorical_columns]

,Neighborhood,BuildingType,PrimaryPropertyType
1008,Northeast,Multifamily lr (1-4),Low-rise multifamily
1784,North,Multifamily lr (1-4),Low-rise multifamily
1563,Lake union,Multifamily lr (1-4),Low-rise multifamily
452,Magnolia / queen anne,Nonresidential,Small- and mid-sized office
1561,Downtown,Multifamily hr (10+),High-rise multifamily
...,...,...,...
2763,East,Multifamily mr (5-9),Mid-rise multifamily
905,Southwest,Nonresidential,Small- and mid-sized office
1096,Northwest,Multifamily lr (1-4),Low-rise multifamily
235,Downtown,Nonresidential,Small- and mid-sized office


##### Encoding the categorical features of the train set


In [334]:
print("Now, we can use the One Hot Encoder.")
print("With the one hot encoder, we will get :", sum([train[categorical_columns].nunique()[i] for i in range(len(categorical_columns))]), "columns to encodes the categorical features.")

Now, we can use the One Hot Encoder.
With the one hot encoder, we will get : 43 columns to encodes the categorical features.


In [335]:
# 0) creating instance of one-hot-encoder
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False) # if sparse=True (by default), we need to add .toarray() to encoded_categorical_data

# 1) Fit the encoder on the training set
one_hot_encoder.fit(train_std[categorical_columns])

# 2) we get the encoded numpy array
encoded_categorical_data = one_hot_encoder.transform(train_std[categorical_columns])

# 3) we make a list of the columns names
encoded_categorical_data_names = one_hot_encoder.get_feature_names_out().tolist()
print("We have indeed :", len(encoded_categorical_data_names), "labels after encoding the categorical variables.")

# 4) we recreate a dataframe with the column names and the numpy array
train_encoded = pd.DataFrame(columns=encoded_categorical_data_names,
                               data=encoded_categorical_data,
                               index=train_std.index)
display(train_encoded.sort_index())

We have indeed : 43 labels after encoding the categorical variables.


,Neighborhood_Ballard,Neighborhood_Central,Neighborhood_Delridge,Neighborhood_Downtown,Neighborhood_East,Neighborhood_Greater duwamish,Neighborhood_Lake union,Neighborhood_Magnolia / queen anne,Neighborhood_North,Neighborhood_Northeast,...,PrimaryPropertyType_Residence hall,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Retail store,PrimaryPropertyType_Self-storage facility,PrimaryPropertyType_Senior care community,PrimaryPropertyType_Small- and mid-sized office,PrimaryPropertyType_Supermarket / grocery store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship facility
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3139,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [336]:
# 5) Concatenate the two dataframes for the training set

print("We need to add YearBuilt to the list of features.")
numerical_columns.tolist()
features_to_merge = numerical_columns.tolist().copy()
features_to_merge.append("YearBuilt")
features_to_merge.extend(models_energy_target)###
print(features_to_merge, end="\n\n")

print("ASK JEREMY : merge based on index ok ? or should I put back OSEBuildingID ??")
train_std_encoded = pd.merge(train_std[features_to_merge].sort_index(), train_encoded.sort_index(), left_index=True, right_index=True)
display(train_std_encoded.sort_index())

We need to add YearBuilt to the list of features.
['Log-NumberofBuildings', 'Log-NumberofFloors', 'Log-PropertyGFATotal', 'Log-PropertyGFAParking', 'Log-SecondLargestPropertyUseTypeGFA', 'Log-ThirdLargestPropertyUseTypeGFA', 'YearBuilt', 'Log-SteamUse(kBtu)', 'Log-Electricity(kBtu)', 'Log-NaturalGas(kBtu)', 'Log-TotalEnergy(kBtu)']

ASK JEREMY : merge based on index ok ? or should I put back OSEBuildingID ??


,Log-NumberofBuildings,Log-NumberofFloors,Log-PropertyGFATotal,Log-PropertyGFAParking,Log-SecondLargestPropertyUseTypeGFA,Log-ThirdLargestPropertyUseTypeGFA,YearBuilt,Log-SteamUse(kBtu),Log-Electricity(kBtu),Log-NaturalGas(kBtu),...,PrimaryPropertyType_Residence hall,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Retail store,PrimaryPropertyType_Self-storage facility,PrimaryPropertyType_Senior care community,PrimaryPropertyType_Small- and mid-sized office,PrimaryPropertyType_Supermarket / grocery store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship facility
0,-0.035067,1.739835,0.529688,-0.405348,-0.923528,-0.439239,1927,14.510597,15.188220,14.059596,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.035067,1.602172,0.712666,2.274807,1.072892,2.153810,1996,0.000000,14.991964,15.453552,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.035067,3.756773,3.287491,2.990638,-0.923528,-0.439239,1969,16.886654,17.718022,14.216834,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,-0.035067,-0.782088,0.640225,2.526634,-0.923528,-0.439239,1999,0.000000,15.813123,15.366298,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,-0.035067,1.107393,0.703626,-0.405348,-0.923528,-0.439239,1926,14.638056,15.106575,12.861492,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,-0.035067,0.096471,-0.349445,-0.405348,0.841584,-0.439239,2015,0.000000,13.513113,12.975567,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3139,-0.035067,-0.287309,-0.913535,1.379140,0.866893,2.071010,2015,0.000000,13.679047,14.085077,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3140,-0.035067,0.904820,0.151911,1.959102,-0.923528,-0.439239,2015,0.000000,14.223590,13.633661,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3141,-0.035067,0.904820,2.037475,-0.405348,-0.923528,-0.439239,2015,0.000000,15.991918,15.850596,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Encoding the categorical features of the test set

In [337]:
# 5) One Hot Encoding on the testing set

# 5.1) we get the encoded numpy array
TEST_encoded_categorical_data = one_hot_encoder.transform(test_std[categorical_columns])

print("ASK JEREMY for this method below ???")
# 5.2) we recreate a dataframe with the column names and the numpy array
test_encoded = pd.DataFrame(columns=encoded_categorical_data_names,
                               data=TEST_encoded_categorical_data,
                               index=test_std.index)
display(test_encoded.sort_index())

print("ASK JEREMY : merge based on index ok ? or should I put back OSEBuildingID ??")
test_std_encoded = pd.merge(test_std[features_to_merge].sort_index(), test_encoded.sort_index(), left_index=True, right_index=True)
display(test_std_encoded.sort_index())

ASK JEREMY for this method below ???


,Neighborhood_Ballard,Neighborhood_Central,Neighborhood_Delridge,Neighborhood_Downtown,Neighborhood_East,Neighborhood_Greater duwamish,Neighborhood_Lake union,Neighborhood_Magnolia / queen anne,Neighborhood_North,Neighborhood_Northeast,...,PrimaryPropertyType_Residence hall,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Retail store,PrimaryPropertyType_Self-storage facility,PrimaryPropertyType_Senior care community,PrimaryPropertyType_Small- and mid-sized office,PrimaryPropertyType_Supermarket / grocery store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship facility
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3142,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3143,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


ASK JEREMY : merge based on index ok ? or should I put back OSEBuildingID ??


,Log-NumberofBuildings,Log-NumberofFloors,Log-PropertyGFATotal,Log-PropertyGFAParking,Log-SecondLargestPropertyUseTypeGFA,Log-ThirdLargestPropertyUseTypeGFA,YearBuilt,Log-SteamUse(kBtu),Log-Electricity(kBtu),Log-NaturalGas(kBtu),...,PrimaryPropertyType_Residence hall,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Retail store,PrimaryPropertyType_Self-storage facility,PrimaryPropertyType_Senior care community,PrimaryPropertyType_Small- and mid-sized office,PrimaryPropertyType_Supermarket / grocery store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship facility
3,-0.035067,1.452522,0.105529,-0.405348,-0.923528,-0.439239,1926,14.610513,14.833970,14.409508,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.035067,2.392512,1.324188,2.668961,1.385691,-0.439239,1980,0.000000,15.496079,15.990717,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-0.035067,1.602172,0.456337,-0.405348,-0.923528,-0.439239,1926,0.000000,14.849128,14.896495,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,-0.035067,1.602172,1.165954,2.343535,-0.923528,-0.439239,1969,15.471291,15.638062,15.339794,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,-0.035067,1.288600,0.173204,-0.405348,-0.923528,-0.439239,1928,14.320129,14.110417,15.095529,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,-0.035067,0.904820,1.553162,2.618557,1.527976,-0.439239,2015,0.000000,14.946797,15.057593,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3142,-0.035067,0.675162,0.497963,2.128455,0.944811,1.780089,2015,0.000000,14.586193,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3143,-0.035067,0.675162,2.107630,2.756954,1.478762,3.044854,2015,0.000000,16.978316,16.570909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3144,-0.035067,0.904820,1.521235,2.654283,1.298680,-0.439239,2015,0.000000,14.503028,14.262382,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [338]:
print("We can start now the modelling to predict the feature wanted.")

display(train_std_encoded)
display(test_std_encoded)


We can start now the modelling to predict the feature wanted.


,Log-NumberofBuildings,Log-NumberofFloors,Log-PropertyGFATotal,Log-PropertyGFAParking,Log-SecondLargestPropertyUseTypeGFA,Log-ThirdLargestPropertyUseTypeGFA,YearBuilt,Log-SteamUse(kBtu),Log-Electricity(kBtu),Log-NaturalGas(kBtu),...,PrimaryPropertyType_Residence hall,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Retail store,PrimaryPropertyType_Self-storage facility,PrimaryPropertyType_Senior care community,PrimaryPropertyType_Small- and mid-sized office,PrimaryPropertyType_Supermarket / grocery store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship facility
0,-0.035067,1.739835,0.529688,-0.405348,-0.923528,-0.439239,1927,14.510597,15.188220,14.059596,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.035067,1.602172,0.712666,2.274807,1.072892,2.153810,1996,0.000000,14.991964,15.453552,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.035067,3.756773,3.287491,2.990638,-0.923528,-0.439239,1969,16.886654,17.718022,14.216834,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,-0.035067,-0.782088,0.640225,2.526634,-0.923528,-0.439239,1999,0.000000,15.813123,15.366298,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,-0.035067,1.107393,0.703626,-0.405348,-0.923528,-0.439239,1926,14.638056,15.106575,12.861492,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,-0.035067,0.096471,-0.349445,-0.405348,0.841584,-0.439239,2015,0.000000,13.513113,12.975567,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3139,-0.035067,-0.287309,-0.913535,1.379140,0.866893,2.071010,2015,0.000000,13.679047,14.085077,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3140,-0.035067,0.904820,0.151911,1.959102,-0.923528,-0.439239,2015,0.000000,14.223590,13.633661,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3141,-0.035067,0.904820,2.037475,-0.405348,-0.923528,-0.439239,2015,0.000000,15.991918,15.850596,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Log-NumberofBuildings,Log-NumberofFloors,Log-PropertyGFATotal,Log-PropertyGFAParking,Log-SecondLargestPropertyUseTypeGFA,Log-ThirdLargestPropertyUseTypeGFA,YearBuilt,Log-SteamUse(kBtu),Log-Electricity(kBtu),Log-NaturalGas(kBtu),...,PrimaryPropertyType_Residence hall,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Retail store,PrimaryPropertyType_Self-storage facility,PrimaryPropertyType_Senior care community,PrimaryPropertyType_Small- and mid-sized office,PrimaryPropertyType_Supermarket / grocery store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship facility
3,-0.035067,1.452522,0.105529,-0.405348,-0.923528,-0.439239,1926,14.610513,14.833970,14.409508,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.035067,2.392512,1.324188,2.668961,1.385691,-0.439239,1980,0.000000,15.496079,15.990717,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-0.035067,1.602172,0.456337,-0.405348,-0.923528,-0.439239,1926,0.000000,14.849128,14.896495,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,-0.035067,1.602172,1.165954,2.343535,-0.923528,-0.439239,1969,15.471291,15.638062,15.339794,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,-0.035067,1.288600,0.173204,-0.405348,-0.923528,-0.439239,1928,14.320129,14.110417,15.095529,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,-0.035067,0.904820,1.553162,2.618557,1.527976,-0.439239,2015,0.000000,14.946797,15.057593,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3142,-0.035067,0.675162,0.497963,2.128455,0.944811,1.780089,2015,0.000000,14.586193,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3143,-0.035067,0.675162,2.107630,2.756954,1.478762,3.044854,2015,0.000000,16.978316,16.570909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3144,-0.035067,0.904820,1.521235,2.654283,1.298680,-0.439239,2015,0.000000,14.503028,14.262382,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [339]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [340]:
print("We rename X_train_std_encoded to X_train, the same for X_test.")
train_preprocess = train_std_encoded.copy()
test_preprocess = test_std_encoded.copy()

train_preprocess
test_preprocess

We rename X_train_std_encoded to X_train, the same for X_test.


,Log-NumberofBuildings,Log-NumberofFloors,Log-PropertyGFATotal,Log-PropertyGFAParking,Log-SecondLargestPropertyUseTypeGFA,Log-ThirdLargestPropertyUseTypeGFA,YearBuilt,Log-SteamUse(kBtu),Log-Electricity(kBtu),Log-NaturalGas(kBtu),...,PrimaryPropertyType_Residence hall,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Retail store,PrimaryPropertyType_Self-storage facility,PrimaryPropertyType_Senior care community,PrimaryPropertyType_Small- and mid-sized office,PrimaryPropertyType_Supermarket / grocery store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship facility
3,-0.035067,1.452522,0.105529,-0.405348,-0.923528,-0.439239,1926,14.610513,14.833970,14.409508,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.035067,2.392512,1.324188,2.668961,1.385691,-0.439239,1980,0.000000,15.496079,15.990717,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-0.035067,1.602172,0.456337,-0.405348,-0.923528,-0.439239,1926,0.000000,14.849128,14.896495,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,-0.035067,1.602172,1.165954,2.343535,-0.923528,-0.439239,1969,15.471291,15.638062,15.339794,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,-0.035067,1.288600,0.173204,-0.405348,-0.923528,-0.439239,1928,14.320129,14.110417,15.095529,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,-0.035067,0.904820,1.553162,2.618557,1.527976,-0.439239,2015,0.000000,14.946797,15.057593,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3142,-0.035067,0.675162,0.497963,2.128455,0.944811,1.780089,2015,0.000000,14.586193,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3143,-0.035067,0.675162,2.107630,2.756954,1.478762,3.044854,2015,0.000000,16.978316,16.570909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3144,-0.035067,0.904820,1.521235,2.654283,1.298680,-0.439239,2015,0.000000,14.503028,14.262382,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [341]:
# Sauvegarde des jeux de données
train_preprocess.to_csv(r'C:\Users\oumei\Documents\OC_projets\P4\P4_Data_Science_OpenClassrooms\other_documents\Example\output\try\train_energy.csv', encoding='utf-8', index=False)
test_preprocess.to_csv(r'C:\Users\oumei\Documents\OC_projets\P4\P4_Data_Science_OpenClassrooms\other_documents\Example\output\try\test_energy.csv', encoding='utf-8', index=False)
